In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet(r"C:\Users\rtc32\OneDrive\바탕 화면\final_project\project\train_all.parquet")

In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [5]:
from sklearn.preprocessing import LabelEncoder
df_converted = df.copy()  # 원본 보존

# 1. datetime 컬럼 → int64로 변환
for col in df_converted.select_dtypes(include=['datetime64[ns]']).columns:
    df_converted[col] = df_converted[col].astype('int64')

# 2. object 컬럼 → LabelEncoder로 변환
for col in df_converted.select_dtypes(include=['object']).columns:
    try:
        le = LabelEncoder()
        df_converted[col] = le.fit_transform(df_converted[col].astype(str))  # NaN 포함 가능성 방지
    except Exception as e:
        print(f"⚠️ 컬럼 '{col}' 인코딩 중 오류 발생: {e}")

In [11]:

for col in df_converted.columns:
    print(f"컬럼명: {col}")
    print(f" - 데이터 타입: {df_converted[col].dtype}")
    print(f" - 고유값(최대 10개): {df_converted[col].unique()[:10]}")
    print("-" * 40)

컬럼명: ID
 - 데이터 타입: int32
 - 고유값(최대 10개): [0 1 2 3 4 5 6 7 8 9]
----------------------------------------
컬럼명: 기준년월
 - 데이터 타입: int64
 - 고유값(최대 10개): [201807 201808 201809 201810 201811 201812]
----------------------------------------
컬럼명: Segment
 - 데이터 타입: int32
 - 고유값(최대 10개): [3 4 2 0 1]
----------------------------------------
컬럼명: 이용금액_R3M_신용체크
 - 데이터 타입: int64
 - 고유값(최대 10개): [   196  13475  23988   3904   1190      0  10571   3463 124967  21001]
----------------------------------------
컬럼명: 이용금액_R3M_신용
 - 데이터 타입: int64
 - 고유값(최대 10개): [   196  13475  23988   3904      0  10571   3463 121279  21001  20212]
----------------------------------------
컬럼명: _1순위카드이용금액
 - 데이터 타입: int64
 - 고유값(최대 10개): [ 3681 13323 24493  5933     0 10378  1062 68078 18796  9422]
----------------------------------------
컬럼명: _2순위카드이용금액
 - 데이터 타입: int64
 - 고유값(최대 10개): [    0  3196 38051  9942 23326  8190  1798  4184  8469  5346]
----------------------------------------
컬럼명: 이용금액_일시불_B0M
 - 데이터 타입: int32
 -

In [8]:
df_converted.dtypes

ID                 int32
기준년월               int64
Segment            int32
이용금액_R3M_신용체크      int64
이용금액_R3M_신용        int64
_1순위카드이용금액         int64
_2순위카드이용금액         int64
이용금액_일시불_B0M       int32
이용건수_신용_R12M       int32
이용건수_신판_R12M       int32
이용건수_일시불_R12M      int32
이용금액_일시불_R12M      int32
최대이용금액_일시불_R12M    int32
이용금액_일시불_R6M       int32
이용금액_일시불_R3M       int32
_1순위업종_이용금액        int32
_2순위업종_이용금액        int32
_3순위업종_이용금액        int32
_1순위쇼핑업종_이용금액      int32
_2순위쇼핑업종_이용금액      int32
이용금액_오프라인_R6M      int32
이용금액_오프라인_R3M      int32
이용금액_오프라인_B0M      int32
정상청구원금_B0M         int32
정상입금원금_B0M         int32
정상청구원금_B2M         int32
정상입금원금_B2M         int32
정상청구원금_B5M         int32
정상입금원금_B5M         int32
이용금액대              int32
청구금액_R6M           int64
청구금액_R3M           int64
청구금액_B0            int64
평잔_일시불_6M          int64
월중평잔_일시불_B0M       int64
월중평잔_일시불           int64
평잔_일시불_3M          int64
잔액_일시불_B0M         int64
dtype: object

In [15]:
df = df_converted.drop(columns=['기준년월', "ID", "Segment"])

In [16]:
# 상관계수로 선택된 변수만 사용
X = df

# 상수항 추가 (intercept term)
X = add_constant(X)

# VIF 계산
vif_df = pd.DataFrame()
vif_df["변수명"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# 결과 출력 (상수항 제외)
print(vif_df[vif_df["변수명"] != "const"])

                변수명           VIF
1     이용금액_R3M_신용체크      9.622707
2       이용금액_R3M_신용     88.312531
3        _1순위카드이용금액     52.518245
4        _2순위카드이용금액     12.729996
5      이용금액_일시불_B0M     56.813094
6      이용건수_신용_R12M  13316.366911
7      이용건수_신판_R12M  15031.001951
8     이용건수_일시불_R12M   3028.606699
9     이용금액_일시불_R12M     19.938680
10  최대이용금액_일시불_R12M      4.405540
11     이용금액_일시불_R6M     53.859971
12     이용금액_일시불_R3M     84.534472
13      _1순위업종_이용금액      5.834593
14      _2순위업종_이용금액      5.815241
15      _3순위업종_이용금액      6.399600
16    _1순위쇼핑업종_이용금액      2.958891
17    _2순위쇼핑업종_이용금액      5.676972
18    이용금액_오프라인_R6M      9.998201
19    이용금액_오프라인_R3M     11.292866
20    이용금액_오프라인_B0M     10.858182
21       정상청구원금_B0M     32.571054
22       정상입금원금_B0M     17.734041
23       정상청구원금_B2M     29.541772
24       정상입금원금_B2M     15.601714
25       정상청구원금_B5M     18.656709
26       정상입금원금_B5M     11.409215
27            이용금액대      4.965438
28         청구금액_R6M     13.061072
29         청구금

In [18]:
vif_filtered = vif_df[vif_df['VIF'] <= 10]
print(vif_filtered)

                변수명       VIF
1     이용금액_R3M_신용체크  9.622707
10  최대이용금액_일시불_R12M  4.405540
13      _1순위업종_이용금액  5.834593
14      _2순위업종_이용금액  5.815241
15      _3순위업종_이용금액  6.399600
16    _1순위쇼핑업종_이용금액  2.958891
17    _2순위쇼핑업종_이용금액  5.676972
18    이용금액_오프라인_R6M  9.998201
27            이용금액대  4.965438
